<a href="https://colab.research.google.com/github/iguv221/Tobigs_Assignment/blob/main/4%EC%A3%BC%EC%B0%A8_KNN_%EA%B9%80%EC%84%B1%EC%9A%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 17기 KNN 정규세션 과제

## KNN 구현해보기
### 1. Preprocssing / EDA
지금까지 배운 내용을 토대로 해당 데이터에 대해 자유롭게 전처리와 EDA를 진행해주세요.
### 2. KNN 구현 & 파라미터 튜닝
수업 내용 및 실습 자료를 참고하여 KNN을 구현하고 파라미터 튜닝을 하며 결과를 비교해주세요.
### 3. Evaluation
결과에 대한 평가를 진행하고, 나름의 해석을 달아주세요.

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/ToBigs/과제 폴더/4주차/Tobigs18기_Week4_KNN&Clustering/과제/blackfriday.csv", index_col = 0)
print(len(df))
df

4998


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780
...,...,...,...,...,...,...,...,...,...,...,...,...
4993,1000837,P00147942,M,55+,7,C,1,1,1,16.0,NaN,11701
4994,1000293,P00358242,M,55+,1,C,1,1,8,NaN,NaN,2045
4995,1001908,P00173342,M,55+,13,C,3,1,1,6.0,14.0,4505
4996,1005517,P00336742,M,55+,0,C,2,1,12,NaN,NaN,1413


# 데이터의 개수는 4998, 거의 5000 이다.

# Step 1.
# 데이터 전처리

In [4]:
### 결측치 확인
df.isna().sum()

User_ID                          0
Product_ID                       0
Gender                           0
Age                              0
Occupation                       0
City_Category                    0
Stay_In_Current_City_Years       0
Marital_Status                   0
Product_Category_1               0
Product_Category_2            1533
Product_Category_3            3454
Purchase                         0
dtype: int64

In [5]:
### Product_Category_2 와 Product_Category_3 은 5000개 전체 데이터에 비해 결측치가 너무 많다.
### 물품의 분류는 가격에 영향을 끼칠 것이라 생각은 들지만 이 정도 결측치로 예측을 진행하기에는 힘들기에 제거한다.
df = df.drop(columns=['Product_Category_2','Product_Category_3'])

In [6]:
print(df.columns)
for col in df.columns:
    print('\n\n',col)
    print(df[col].value_counts())

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Purchase'],
      dtype='object')


 User_ID
1002063    12
1005795    12
1001980    11
1004344    11
1006036    10
           ..
1004980     1
1004385     1
1004138     1
1002648     1
1000837     1
Name: User_ID, Length: 2656, dtype: int64


 Product_ID
P00265242    21
P00220442    16
P00114942    15
P00010742    15
P00148642    15
             ..
P00072342     1
P00037842     1
P00207042     1
P00129742     1
P00358242     1
Name: Product_ID, Length: 1797, dtype: int64


 Gender
M    3765
F    1233
Name: Gender, dtype: int64


 Age
26-35    1996
36-45    1000
18-25     906
46-50     414
51-55     350
55+       195
0-17      137
Name: Age, dtype: int64


 Occupation
4     706
0     572
7     535
1     460
17    339
12    305
20    285
2     251
14    245
16    244
6     186
3     160
15    133
10    122
5      98
11     95

## 위의 value_counts 을 통해 파악할 수 있는 점.
1. 'User_ID' 와 'Product_ID' 은 제거해도 될것 같다.
2. 'Gender' 은 M 과 F 으로 이루어졌는데 M 은 0으로, F 은 1 로 변환하는게 좋을 것 같다.
3. 'Age', 'City_Category',  은 classified age 으로 one-hot encoding 이 적절해 보인다.
4. 'Stay_In_Current_City_Years' 에서 '4+' 은 4로 바꾼다.
5. 'Occupation', 'Product_Category_1' 은 일단은 그대로 둔다.

In [7]:
### 우선 User_ID 와 Product_ID 은 판매 가격에 영향을 안 끼칠 것이기 때문에 제거.
df = df.drop(columns=['User_ID','Product_ID'])
df

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
0,F,0-17,10,A,3,0,5,2010
1,F,0-17,10,A,1,0,7,4483
2,F,0-17,10,A,1,0,1,7696
3,F,0-17,10,A,3,0,2,16429
4,F,0-17,10,A,3,0,3,5780
...,...,...,...,...,...,...,...,...
4993,M,55+,7,C,1,1,1,11701
4994,M,55+,1,C,1,1,8,2045
4995,M,55+,13,C,3,1,1,4505
4996,M,55+,0,C,2,1,12,1413


In [8]:
df['Gender'] = np.where(df['Gender']=='M',0,1)
df

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
0,1,0-17,10,A,3,0,5,2010
1,1,0-17,10,A,1,0,7,4483
2,1,0-17,10,A,1,0,1,7696
3,1,0-17,10,A,3,0,2,16429
4,1,0-17,10,A,3,0,3,5780
...,...,...,...,...,...,...,...,...
4993,0,55+,7,C,1,1,1,11701
4994,0,55+,1,C,1,1,8,2045
4995,0,55+,13,C,3,1,1,4505
4996,0,55+,0,C,2,1,12,1413


In [9]:
one_hot_columns = ['Age','City_Category']
for col in one_hot_columns:
    dummies = pd.get_dummies(df[col],prefix=col)
    df = df.join(dummies)
    df = df.drop(columns=col)
df

,Gender,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase,Age_0-17,Age_18-25,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Age_55+,City_Category_A,City_Category_B,City_Category_C
0,1,10,3,0,5,2010,1,0,0,0,0,0,0,1,0,0
1,1,10,1,0,7,4483,1,0,0,0,0,0,0,1,0,0
2,1,10,1,0,1,7696,1,0,0,0,0,0,0,1,0,0
3,1,10,3,0,2,16429,1,0,0,0,0,0,0,1,0,0
4,1,10,3,0,3,5780,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,0,7,1,1,1,11701,0,0,0,0,0,0,1,0,0,1
4994,0,1,1,1,8,2045,0,0,0,0,0,0,1,0,0,1
4995,0,13,3,1,1,4505,0,0,0,0,0,0,1,0,0,1
4996,0,0,2,1,12,1413,0,0,0,0,0,0,1,0,0,1


In [10]:
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].replace(['4+'],'4')

In [11]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

In [12]:
y = df['Purchase']
x = df.drop(columns='Purchase')

In [13]:
x

,Gender,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Age_0-17,Age_18-25,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Age_55+,City_Category_A,City_Category_B,City_Category_C
0,1,10,3,0,5,1,0,0,0,0,0,0,1,0,0
1,1,10,1,0,7,1,0,0,0,0,0,0,1,0,0
2,1,10,1,0,1,1,0,0,0,0,0,0,1,0,0
3,1,10,3,0,2,1,0,0,0,0,0,0,1,0,0
4,1,10,3,0,3,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,0,7,1,1,1,0,0,0,0,0,0,1,0,0,1
4994,0,1,1,1,8,0,0,0,0,0,0,1,0,0,1
4995,0,13,3,1,1,0,0,0,0,0,0,1,0,0,1
4996,0,0,2,1,12,0,0,0,0,0,0,1,0,0,1


In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=42)
KNN = KNeighborsRegressor(12)
KNN.fit(x_train, y_train)

KNeighborsRegressor(n_neighbors=12)

In [15]:
train_pred = KNN.predict(x_train)
test_pred = KNN.predict(x_test)

In [18]:
print(np.array(y_train))
print(train_pred)

[ 4497 15210 10147 ...  6909  8112 15365]
[13571.41666667 13180.83333333 12984.         ...  5764.58333333
  6983.58333333 14377.08333333]


In [17]:
print(np.array(y_test))
print(test_pred)

84       6682
2470     5430
2803    11739
4986     8723
4923    11561
        ...  
2644    13293
3711    12601
2031     8847
135     15605
4676     9753
Name: Purchase, Length: 500, dtype: int64
[ 9898.91666667  6943.08333333 14926.91666667  6021.75
 13113.08333333  6139.66666667 12144.5         9557.75
  5347.75        4885.41666667 12612.08333333 14042.25
  6691.83333333 12347.5        12953.          8964.75
  6681.58333333  7321.5         7728.91666667  6201.83333333
  5368.08333333 11801.91666667  9270.5         6071.33333333
  6332.66666667  7435.5        14186.75        7496.83333333
  3135.66666667  7980.83333333 14637.66666667  6265.66666667
 13584.08333333  9720.16666667  7725.91666667  7634.83333333
 13023.25        7322.          6381.66666667  5667.5
  5032.08333333  5201.16666667  5044.75        7015.83333333
  6858.58333333 13305.66666667  6159.91666667 12814.41666667
  9452.58333333  5780.66666667  8679.25        5721.83333333
  6077.75        6701.58333333  6347.5    